In [62]:
# import modules
import os
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

In [4]:
# Import Data (Smaller test data to start with)

file_path = os.path.join('data-society-us-air-pollution-data', 'uspollution', 'pollution_us_2000_2016.csv')
raw_data = pd.read_csv(file_path)

In [7]:
# DataFrame info
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1746661 entries, 0 to 1746660
Data columns (total 29 columns):
Unnamed: 0           int64
State Code           int64
County Code          int64
Site Num             int64
Address              object
State                object
County               object
City                 object
Date Local           object
NO2 Units            object
NO2 Mean             float64
NO2 1st Max Value    float64
NO2 1st Max Hour     int64
NO2 AQI              int64
O3 Units             object
O3 Mean              float64
O3 1st Max Value     float64
O3 1st Max Hour      int64
O3 AQI               int64
SO2 Units            object
SO2 Mean             float64
SO2 1st Max Value    float64
SO2 1st Max Hour     int64
SO2 AQI              float64
CO Units             object
CO Mean              float64
CO 1st Max Value     float64
CO 1st Max Hour      int64
CO AQI               float64
dtypes: float64(10), int64(10), object(9)
memory usage: 386.5+ MB


In [12]:
raw_data.describe()

,Unnamed: 0,State Code,County Code,Site Num,NO2 Mean,NO2 1st Max Value,NO2 1st Max Hour,NO2 AQI,O3 Mean,O3 1st Max Value,O3 1st Max Hour,O3 AQI,SO2 Mean,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI
count,1.746661e+06,1.746661e+06,1.746661e+06,1.746661e+06,1.746661e+06,1.746661e+06,1.746661e+06,1.746661e+06,1.746661e+06,1.746661e+06,1.746661e+06,1.746661e+06,1.746661e+06,1.746661e+06,1.746661e+06,873754.000000,1.746661e+06,1.746661e+06,1.746661e+06,873338.000000
mean,5.471414e+04,2.230907e+01,7.169381e+01,1.118214e+03,1.282193e+01,2.541485e+01,1.173102e+01,2.389822e+01,2.612485e-02,3.920331e-02,1.017053e+01,3.605012e+01,1.870364e+00,4.492185e+00,9.664906e+00,7.115945,3.682177e-01,6.201067e-01,7.875026e+00,5.996595
std,3.372908e+04,1.725621e+01,7.948023e+01,2.003103e+03,9.504814e+00,1.599963e+01,7.877501e+00,1.516280e+01,1.136974e-02,1.534362e-02,4.003144e+00,1.978042e+01,2.760435e+00,7.679866e+00,6.731228e+00,11.937473,3.140231e-01,6.439361e-01,7.978844e+00,5.851836
min,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-2.000000e+00,-2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-2.000000e+00,-2.000000e+00,0.000000e+00,0.000000,-4.375000e-01,-4.000000e-01,0.000000e+00,0.000000
25%,2.575300e+04,6.000000e+00,1.700000e+01,9.000000e+00,5.750000e+00,1.300000e+01,5.000000e+00,1.200000e+01,1.787500e-02,2.900000e-02,9.000000e+00,2.500000e+01,2.565220e-01,8.000000e-01,5.000000e+00,1.000000,1.834580e-01,2.920000e-01,0.000000e+00,2.000000
50%,5.304500e+04,1.700000e+01,5.900000e+01,6.000000e+01,1.073913e+01,2.400000e+01,9.000000e+00,2.300000e+01,2.587500e-02,3.800000e-02,1.000000e+01,3.300000e+01,9.875000e-01,2.000000e+00,8.000000e+00,3.000000,2.926250e-01,4.000000e-01,6.000000e+00,5.000000
75%,8.033600e+04,4.000000e+01,9.700000e+01,1.039000e+03,1.771364e+01,3.570000e+01,2.000000e+01,3.300000e+01,3.391700e-02,4.800000e-02,1.100000e+01,4.200000e+01,2.325000e+00,5.000000e+00,1.400000e+01,9.000000,4.666670e-01,8.000000e-01,1.300000e+01,8.000000
max,1.345750e+05,8.000000e+01,6.500000e+02,9.997000e+03,1.395417e+02,2.670000e+02,2.300000e+01,1.320000e+02,9.508300e-02,1.410000e-01,2.300000e+01,2.180000e+02,3.216250e+02,3.510000e+02,2.300000e+01,200.000000,7.508333e+00,1.990000e+01,2.300000e+01,201.000000


In [14]:
raw_data.columns

Index(['Unnamed: 0', 'State Code', 'County Code', 'Site Num', 'Address',
       'State', 'County', 'City', 'Date Local', 'NO2 Units', 'NO2 Mean',
       'NO2 1st Max Value', 'NO2 1st Max Hour', 'NO2 AQI', 'O3 Units',
       'O3 Mean', 'O3 1st Max Value', 'O3 1st Max Hour', 'O3 AQI', 'SO2 Units',
       'SO2 Mean', 'SO2 1st Max Value', 'SO2 1st Max Hour', 'SO2 AQI',
       'CO Units', 'CO Mean', 'CO 1st Max Value', 'CO 1st Max Hour', 'CO AQI'],
      dtype='object')

In [31]:
# Data cleaning (any zeros or NaN)
col_list = ['State', 'Date Local', 'NO2 Mean', 'NO2 1st Max Value', 'O3 Mean', 'O3 1st Max Value', 'SO2 Mean', 'SO2 1st Max Value', 'CO Mean',  'CO 1st Max Value']

trim_data = raw_data[col_list]

for col in col_list:
    print(col)
    display(trim_data.loc[trim_data[col].isna()])


State


,State,Date Local,NO2 Mean,NO2 1st Max Value,O3 Mean,O3 1st Max Value,SO2 Mean,SO2 1st Max Value,CO Mean,CO 1st Max Value


Date Local


,State,Date Local,NO2 Mean,NO2 1st Max Value,O3 Mean,O3 1st Max Value,SO2 Mean,SO2 1st Max Value,CO Mean,CO 1st Max Value


NO2 Mean


...

,State,Date Local,NO2 Mean,NO2 1st Max Value,O3 Mean,O3 1st Max Value,SO2 Mean,SO2 1st Max Value,CO Mean,CO 1st Max Value


CO Mean


,State,Date Local,NO2 Mean,NO2 1st Max Value,O3 Mean,O3 1st Max Value,SO2 Mean,SO2 1st Max Value,CO Mean,CO 1st Max Value


CO 1st Max Value


,State,Date Local,NO2 Mean,NO2 1st Max Value,O3 Mean,O3 1st Max Value,SO2 Mean,SO2 1st Max Value,CO Mean,CO 1st Max Value


In [75]:
# Create Tidy Dataset 
tidy_data = trim_data.copy()

# Add Year Column
tidy_data['year'] = tidy_data['Date Local'].str[:4]

# Drop 2016 Data due to partial year
tidy_data = tidy_data.loc[tidy_data['year'] != '2016']

# Save Tidy Data
tidy_data.to_csv('air_pollution_tidy.csv', index = False)

tidy_data.tail()


,State,Date Local,NO2 Mean,NO2 1st Max Value,O3 Mean,O3 1st Max Value,SO2 Mean,SO2 1st Max Value,CO Mean,CO 1st Max Value,year
1722052,Wyoming,2015-12-30,7.047826,18.8,0.036917,0.043,-0.057143,0.2,0.154167,0.200,2015
1722053,Wyoming,2015-12-31,2.445833,8.8,0.038263,0.040,-0.187500,-0.1,0.103458,0.155,2015
1722054,Wyoming,2015-12-31,2.445833,8.8,0.038263,0.040,-0.187500,-0.1,0.100000,0.100,2015
1722055,Wyoming,2015-12-31,2.445833,8.8,0.038263,0.040,-0.175000,-0.1,0.103458,0.155,2015
1722056,Wyoming,2015-12-31,2.445833,8.8,0.038263,0.040,-0.175000,-0.1,0.100000,0.100,2015


In [42]:
# Year-by-year data
grouped_Data = wip_df.groupby(['year']).mean()

In [43]:
grouped_Data

,NO2 Mean,NO2 1st Max Value,O3 Mean,O3 1st Max Value,SO2 Mean,SO2 1st Max Value,CO Mean,CO 1st Max Value
year,,,,,,,,
2000,17.433768,32.844986,0.024374,0.039143,3.635001,8.706905,0.555605,1.002820
2001,17.601948,33.072649,0.025387,0.040566,3.120901,7.863073,0.506644,0.929252
2002,16.385855,31.014175,0.026819,0.041767,2.729866,6.724265,0.498662,0.881102
2003,15.864934,29.947360,0.025527,0.039844,2.851912,6.955952,0.490559,0.850875
2004,14.975201,28.494426,0.024901,0.038773,2.841329,6.879984,0.446285,0.764395
2005,15.010923,29.000439,0.025742,0.040385,2.900743,6.901690,0.415658,0.715437
2006,14.531508,28.559908,0.025937,0.039971,2.596110,6.122259,0.413375,0.726352
2007,13.285443,26.422592,0.026379,0.040197,2.341500,5.578362,0.373295,0.617226
2008,12.240446,24.874386,0.026093,0.039431,1.894672,4.527777,0.328974,0.549430


In [ ]:
# Analysis

In [ ]:
# Visualization

In [1]:
# Conclusion